In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ipc-recommandation/secs.jsonl
/kaggle/input/ipc-recommandation/schemas.json
/kaggle/input/ipc-recommandation/dev.jsonl
/kaggle/input/ipc-recommandation/test.jsonl
/kaggle/input/ipc-recommandation/label_tree.json
/kaggle/input/ipc-recommandation/citation_network.json
/kaggle/input/ipc-recommandation/ils2v.bin
/kaggle/input/ipc-recommandation/type_map.json
/kaggle/input/ipc-recommandation/label_vocab.json
/kaggle/input/ipc-recommandation/best_model.pt


In [2]:
# Install dependencies
!apt-get install -y git make g++

# Clone the Sent2Vec repository
!git clone https://github.com/epfml/sent2vec.git

# Navigate into the repository
%cd sent2vec

# Compile the code
!make

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-email git-gui gitk gitweb git-cvs
  git-mediawiki git-svn
The following packages will be upgraded:
  git
1 upgraded, 0 newly installed, 0 to remove and 128 not upgraded.
Need to get 3,165 kB of archives.
After this operation, 4,096 B of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 git amd64 1:2.34.1-1ubuntu1.12 [3,165 kB]
Fetched 3,165 kB in 1s (2,259 kB/s)
(Reading database ... 127400 files and directories currently installed.)
Preparing to unpack .../git_1%3a2.34.1-1ubuntu1.12_amd64.deb ...
Unpacking git (1:2.34.1-1ubuntu1.12) over (1:2.34.1-1ubuntu1.11) ...
Setting up git (1:2.34.1-1u

In [ ]:
!pip install .

Processing /kaggle/working/sent2vec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# !git clone https://github.com/Law-AI/LeSICiN.git
!rm -rf /kaggle/working/sent2vec


In [ ]:
import sent2vec
sent2vec.Sent2vecModel()

In [ ]:
!pip install torch_geometric

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.0+cu124.html

In [ ]:
import torch_sparse
import numpy as np # linear algebra
import pandas as pd
import copy
import torch_scatter
import numpy as np
import torch
import numpy as np
import string
import copy
import multiprocessing as mp
from tqdm import tqdm
import json
import pickle as pkl
import torch
import torch_sparse
from tqdm import tqdm
from collections import defaultdict
import torch
import random
from tqdm import tqdm
from functools import partial

In [ ]:
hc = {
	"seed": 42,
	"do_train_dev": False,
	"do_test": True,
	"do_infer": True,
	"vocab_limit": None,
	"vocab_thresh": None,
	"weight_scheme": "tws",
	"tws_thresh": 10.0,
	"train_bs": 64,
	"dev_bs": 512,
	"test_bs": 512,
	"infer_bs": 512,
	"max_segments": 128,
	"max_segment_size": None,
	"num_mpath_samples": 8,
	"hidden_size": 200,
	"opt_lr": 1e-4,
	"opt_wt_decay": 1e-4,
	"sch_factor": 0.2,
	"sch_patience": 3,
	"num_epochs": 2,
	"pthresh": 0.51,
	"thetas": [1,2,3],
	"lambdas": [0.25,0.75],
	"dropout": 0.1
}

In [ ]:
dc = {
	"sec_src": "/kaggle/input/ipc-recommandation/secs.jsonl",
	"train_src": "/kaggle/input/ipc-recommandation/train.jsonl",
	"dev_src": "/kaggle/input/ipc-recommandation/dev.jsonl",
	"test_src": "/kaggle/input/ipc-recommandation/test.jsonl",
	"infer_src": None,
	"sec_cache": "/kaggle/working/secs.pkl",
	"train_cache": "/kaggle/working/train.pkl",
	"dev_cache": "/kaggle/working/dev.pkl",
	"test_cache": "/kaggle/working/test.pkl",
	"infer_cache": "/kaggle/working/infer.pkl",
	"s2v_path": "/kaggle/input/ipc-recommandation/ils2v.bin",
	"type_map": "/kaggle/input/ipc-recommandation/type_map.json",
	"label_tree": "/kaggle/input/ipc-recommandation/label_tree.json",
	"citation_network": "/kaggle/input/ipc-recommandation/citation_network.json",
	"schemas": "/kaggle/input/ipc-recommandation/schemas.json",
	"model_load": "/kaggle/input/ipc-recommandation/best_model.pt",
	"metrics_load": None,
	"model_dump": "/kaggle/input/ipc-recommandation/best_model.pt",
	"dev_metrics_dump": "/kaggle/input/ipc-recommandation/dev_metrics.json",
	"test_metrics_dump": "/kaggle/input/ipc-recommandation/test_metrics.json",
	"infer_trg": None
}

In [ ]:
import torch

class LstmNet(torch.nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.lstm = torch.nn.LSTM(hidden_size, hidden_size // 2, batch_first=True, bidirectional=True)

    def forward(self, inputs, mask=None): # [B, S, H], [B, S]
        mask = mask if mask is not None else torch.ones(inputs.size(0), inputs.size(1), device=inputs.device)
        lengths = mask.sum(dim=-1) # [B,]

        # need to pack inputs before passing to RNN and unpack obtained outputs
        pck_inputs = torch.nn.utils.rnn.pack_padded_sequence(inputs, lengths.cpu(), batch_first=True, enforce_sorted=False)
        pck_hidden_all = self.lstm(pck_inputs)[0]
        hidden_all = torch.nn.utils.rnn.pad_packed_sequence(pck_hidden_all, batch_first=True)[0] # [B, S, H]

        return hidden_all

class AttnNet(torch.nn.Module):
    def __init__(self, hidden_size, drop=0.1):
        super().__init__()

        self.hidden_size = hidden_size
        self.attn_fc = torch.nn.Linear(hidden_size, hidden_size)
        self.context = torch.nn.Parameter(torch.rand(hidden_size))

        self.dropout = torch.nn.Dropout(drop)

    def forward(self, inputs, mask=None, dyn_context=None): # [B, S, H], [B, S], [B, H]

        mask = mask if mask is not None else torch.ones(inputs.size(0), inputs.size(1), device=inputs.device)
        # use static (learned) context vector if dynamic context is unavailable
        context = dyn_context if dyn_context is not None else self.context.expand(inputs.size(0), self.hidden_size) # [B, H]

        act_inputs = torch.tanh(self.dropout(self.attn_fc(inputs)))

        scores = torch.bmm(act_inputs, context.unsqueeze(2)).squeeze(2) # [B, S]
        msk_scores = scores.masked_fill((1 - mask).bool(), -1e-32)
        msk_scores = torch.nn.functional.softmax(msk_scores, dim=1)

        hidden = torch.sum(inputs * msk_scores.unsqueeze(2), dim=1) # [B, H]
        return hidden


In [ ]:

class LSIDataset(torch.utils.data.Dataset):
    def __init__(self, jsonl_file=None, data_list=None):
        super().__init__()

        self.annotated = False
        self.sent_vectorized = False
        self.dataset = []
        if data_list is not None:
            self.dataset = copy.deepcopy(data_list)
            for instance in tqdm(self.dataset, desc="Loading data from list"):
                instance['text'] = np.array(instance['text'])
                if 'labels' in instance:
                    self.annotated = True
                    instance['labels'] = np.array(instance['labels'])

        elif jsonl_file is not None:
            self.dataset = []
            with open(jsonl_file) as fr:
                for line in tqdm(fr, desc="Loading data from file"):
                    doc = json.loads(line)
                    text = np.array([sent for sent in doc['text']])
                    newdoc = {'id': doc['id'], 'text': text}
                    if 'labels' in doc:
                        self.annotated = True
                        labels = np.array(doc['labels'])
                        newdoc['labels'] = labels
                    self.dataset.append(newdoc)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return self.dataset[index]

    def save_data(self, data_file):
        # if not os.path.exists(data_file):

        with open(data_file, 'wb') as fw:
            pkl.dump(self, fw)

    def load_data(data_file):
        with open(data_file, 'rb') as fr:
            return pkl.load(fr)

    # remove puncutations and empty sentences
    def preprocess(self):
        for i, instance in enumerate(tqdm(self.dataset, desc="Preprocessing")):
            text = []
            # print(f"{instance['text']} helllllllll {len(instance['text'])}")
            for j, sent in enumerate(instance['text']):
                ppsent = sent.strip().lower().translate(str.maketrans('', '', string.punctuation))
                # print(ppsent)
                if len(ppsent.split()) > 1:
                    text.append(ppsent)
            instance['text'] = np.array(text)

    # break each sentence string into word tokens
    def tokenize(self):
        for i, instance in enumerate(tqdm(self.dataset, desc="Tokenizing")):
            text = []
            for j, sent in enumerate(instance['text']):
                toksent = np.array(sent.strip().split())
                text.append(toksent)
            instance['text'] = np.array(text, dtype=object)

    # generate a vector for each sentence using Sent2Vec
    def sent_vectorize(self, sent2vec_model):
        # print(self.dataset)
        for i, instance in enumerate(tqdm(self.dataset, desc="Embedding sentences")):
            esents = sent2vec_model.embed_sentences(instance['text'])
            instance['text'] = np.delete(esents, np.where(esents.sum(axis=1) == 0)[0], axis=0)
        self.sent_vectorized = True


# unified code for generating mini batches of data for both facts and sections during train / dev / test / inference
class MiniBatch:
    def __init__(self, examples, vocab=None, label_vocab=None, schemas=None, type_map=None, node_vocab=None, edge_vocab=None, adjacency=None, hidden_size=200, max_segments=4, max_segment_size=8, num_mpath_samples=2):
        # provide vocab if not sent vectorized, None otherwise
        self.sent_vectorized = True if vocab is None else False
        # provide label_vocab if annotated, None otherwise
        # self.annotated = True if label_vocab is not None else False
        self.annotated =  False
        # provide graph data if struct encoder is to be used on these examples, None otherwise
        self.sample_metapaths = True if schemas is not None else False

        self.max_segments = max_segments

        if not self.sent_vectorized:
            self.vocab = vocab
            self.max_segment_size = max_segment_size
        else:
            self.sent_hidden_size = hidden_size

        if self.annotated:
            self.label_vocab = label_vocab

        if self.sample_metapaths:
            self.schemas = schemas
            self.type_map = type_map
            self.node_vocab = node_vocab
            self.edge_vocab = edge_vocab
            self.adjacency = adjacency
            self.num_mpath_samples = num_mpath_samples

        max_len = max([len(d['text']) for d in examples])
        max_segments = min(self.max_segments, max_len)

        # expected shape of text tensors
        if not self.sent_vectorized:
            max_segment_size = min(self.max_segment_size, max([len(s) for d in examples for s in d['text']]))
            self.tokens = torch.zeros(len(examples), max_segments, max_segment_size, dtype=torch.long) # [D, S, W]
        else:
            self.doc_inputs = torch.zeros(len(examples), max_segments, self.sent_hidden_size) # [D, S, H]

        self.example_ids = []

        if self.annotated:
            # expected shape of true label indicator tensors
            self.labels = torch.zeros(len(examples), len(self.label_vocab)) # [D, C]

        for i, instance in enumerate(examples):
            if not self.sent_vectorized:
                for j, sent in enumerate(instance['text']):
                    # fill up the j-th sentence of i-th example with word tokens
                    self.tokens[i, j, :len(sent)] = torch.from_numpy(np.array([self.vocab[w] for w in sent]))
            else:
                # fill up the i-th example with sentence embeddings
                self.doc_inputs[i, :len(instance['text']), :] = torch.from_numpy(instance['text'])[:max_segments]

            self.example_ids.append(instance['id'])

            if self.annotated:
                label_list = torch.from_numpy(np.array([self.label_vocab[l] for l in instance['labels']]))
                self.labels[i].scatter_(0, label_list, 1.)

        if not self.sent_vectorized:
            self.mask = (self.tokens != 0).float() # [D, S, W]
        else:
            self.mask = (self.doc_inputs != 0).any(dim=2).float() # [D, S]

        if self.sample_metapaths:
            trg_node_tokens = torch.tensor([self.node_vocab[self.type_map[x]][x] for x in self.example_ids])
            self.node_tokens, self.edge_tokens = self.generate_metapaths(trg_node_tokens, self.schemas, self.adjacency, self.edge_vocab, num_samples=self.num_mpath_samples) # N * [M, D, L+1], N * [M, D, L]

    # sample metapaths using adjacency matrices
    def generate_metapaths(self, indices, schemas, adjacency, edge_vocab, num_samples=2): # [D,]
        indices = indices.repeat(num_samples) # [M*D,]

        tokens, edge_tokens = [], []

        # repeat over all schemas
        for i in range(len(schemas)):
            ins_tokens, ins_edge_tokens = [indices], []
            for keys in schemas[i]:
                neighbours = adjacency[keys].sample(num_neighbors=1, subset=ins_tokens[-1]).squeeze(1) # [M*D,]
                relations = torch.full(neighbours.shape, edge_vocab[keys[1]], dtype=torch.long) # [M*D,]

                ins_tokens.append(neighbours)
                ins_edge_tokens.append(relations)

            ins_tokens = torch.stack(ins_tokens, dim=1)
            ins_tokens = ins_tokens.view(num_samples, -1, ins_tokens.size(1)) # [M, D, L+1]

            ins_edge_tokens = torch.stack(ins_edge_tokens, dim=1)
            ins_edge_tokens = ins_edge_tokens.view(num_samples, -1, ins_edge_tokens.size(1)) # [M, D, L]

            tokens.append(ins_tokens)
            edge_tokens.append(ins_edge_tokens)

        return tokens, edge_tokens

    # automatic memory pinning for faster cpu to cuda transfer
    def pin_memory(self):
        if not self.sent_vectorized:
            self.tokens.pin_memory()
        else:
            self.doc_inputs.pin_memory()
        self.mask.pin_memory()
        if self.annotated:
            self.labels.pin_memory()
        if self.sample_metapaths:
            for i in range(len(self.node_tokens)):
                self.node_tokens[i].pin_memory()
                self.edge_tokens[i].pin_memory()
        return self

    # transfer pinned cpu tensors to cuda
    def cuda(self, dev='cuda'):
        if not self.sent_vectorized:
            self.tokens = self.tokens.cuda(dev, non_blocking=True)
        else:
            self.doc_inputs = self.doc_inputs.cuda(dev, non_blocking=True)
        self.mask = self.mask.cuda(dev, non_blocking=True)
        if self.annotated:
            self.labels = self.labels.cuda(dev, non_blocking=True)
        if self.sample_metapaths:
            for i in range(len(self.node_tokens)):
                self.node_tokens[i] = self.node_tokens[i].cuda(dev, non_blocking=True)
                self.edge_tokens[i] = self.edge_tokens[i].cuda(dev, non_blocking=True)
        return self

def collate_func(examples, **kwargs):
    return MiniBatch(examples, **kwargs)


In [ ]:
import torch
# from model.basicmodules import LstmNet, AttnNet

class HierAttnNet(torch.nn.Module):
    def __init__(self, hidden_size, vocab_size=None, drop=0.1):
        super().__init__()

        if vocab_size is not None:
            self.word_embedding = torch.nn.Embedding(vocab_size, hidden_size)
            self.sent_lstm = LstmNet(hidden_size)
            self.sent_attn = AttnNet(hidden_size, drop=drop)

        self.doc_lstm = LstmNet(hidden_size)
        self.doc_attn = AttnNet(hidden_size, drop=drop)

    def forward(self, tokens=None, doc_inputs=None, mask=None, sent_dyn_context=None, doc_dyn_context=None): # [B, S, W], [B, S, H], [B, S, W] / [B, S], [B, S, H], [B, H]
        if tokens is not None:
            sent_inputs = self.word_embedding(tokens)

            # flatten to 3-D
            sent_inputs = sent_inputs.view(-1, sent_inputs.size(2), sent_inputs.size(3))
            sent_mask = mask.view(-1, mask.size(2))

            if sent_dyn_context is not None:
                sent_dyn_context = sent_dyn_context.view(-1, sent_dyn_context.size(2))

            sent_hidden_all = self.sent_lstm(sent_inputs, sent_mask)
            sent_hidden = self.sent_attn(sent_hidden_all, sent_mask, dyn_context=sent_dyn_context)

            doc_inputs = sent_hidden.view(tokens.size(0), tokens.size(1), -1)
            doc_mask = (mask.sum(dim=2) > 0).float()
        else:
            doc_mask = mask

        doc_hidden_all = self.doc_lstm(doc_inputs, doc_mask)
        doc_hidden = self.doc_attn(doc_hidden_all, doc_mask, dyn_context=doc_dyn_context)
        return doc_hidden

class MetapathAggrNet(torch.nn.Module):
    def __init__(self, node_vocab_size, edge_vocab_size, hidden_size, drop=0.1, gdel=14.):
        super().__init__()
        self.emb_range = gdel / hidden_size

        self.node_embedding = torch.nn.ModuleDict({ntype: torch.nn.Embedding(num_nodes, hidden_size) for ntype, num_nodes in node_vocab_size.items()})
        for ntype, ntype_weights in self.node_embedding.items():
            ntype_weights.weight.data.uniform_(- self.emb_range, self.emb_range)

        self.scale_fc = torch.nn.ModuleDict({ntype: torch.nn.Linear(hidden_size, hidden_size) for ntype in node_vocab_size})

        self.edge_embedding = torch.nn.Embedding(edge_vocab_size, hidden_size // 2)
        self.edge_embedding.weight.data.uniform_(- self.emb_range, self.emb_range)

        self.intra_attention = AttnNet(2 * hidden_size, drop=drop)

        self.inter_fc = torch.nn.Linear(2 * hidden_size, 2 * hidden_size)
        self.inter_context = torch.nn.Parameter(torch.rand(2 * hidden_size))

        self.output_fc = torch.nn.Linear(2 * hidden_size, hidden_size)

        self.dropout = torch.nn.Dropout(drop)

    # Embed each node index using the node embedding matrix and then scale to generate same sized embeddings for each node type
    def embed_and_scale(self, tokens, edge_tokens, schema): # [B, L+1], [B, L]
        inputs, edge_inputs = [], []

        node_type = schema[0][0]
        node_input = self.dropout(self.node_embedding[node_type](tokens[:, 0])) # [B, H]
        inputs.append(self.dropout(self.scale_fc[node_type](node_input)))

        for i in range(edge_tokens.size(1)):
            node_type = schema[i][2]
            node_input = self.dropout(self.node_embedding[node_type](tokens[:, i+1])) # [B, H]
            inputs.append(self.dropout(self.scale_fc[node_type](node_input)))

            edge_inputs.append(self.dropout(self.edge_embedding(edge_tokens[:, i])))
        inputs = torch.stack(inputs, dim=1) # [B, L+1, H]
        edge_inputs = torch.stack(edge_inputs, dim=1) # [B, L, H]
        return inputs, edge_inputs

    # We are following the official implementation of the RotatE algorithm --- https://github.com/DeepGraphLearning/KnowledgeGraphEmbedding
    def rotational_encoding(self, inputs, edge_inputs): # [B, L+1, H], [B, L, H/2]
        PI = 3.14159265358979323846
        hidden = inputs.clone()
        for i in reversed(range(edge_inputs.size(1))):
            hid_real, hid_imag = torch.chunk(hidden.clone()[:, i+1:, :], 2, dim=2) # [B, L-i, H/2], [B, L-i, H/2]
            inp_real, inp_imag = torch.chunk(inputs[:, i, :], 2, dim=1) # [B, H/2], [B, H/2]

            edge_complex = edge_inputs[:, i, :] / (self.emb_range / PI)
            edge_real, edge_imag = torch.cos(edge_inputs[:, i, :]), torch.sin(edge_inputs[:, i, :]) # [B, H/2], [B, H/2]

            out_real = inp_real.unsqueeze(1) + edge_real.unsqueeze(1) * hid_real - edge_imag.unsqueeze(1) * hid_imag # [B, L-i, H/2]
            out_imag = inp_imag.unsqueeze(1) + edge_imag.unsqueeze(1) * hid_real + edge_real.unsqueeze(1) * hid_imag # [B, L-i, H/2]

            hidden[:, i+1:, :] = torch.cat([out_real, out_imag], dim=2)
        path_lens = 1 + torch.arange(hidden.size(1), device=hidden.device) # [L+1]
        return hidden / path_lens.unsqueeze(0).unsqueeze(2)

    def forward(self, tokens, edge_tokens, schemas, intra_context=None, inter_context=None):
        hidden = []

        # serially perform intra-metapath aggregation across the different schemas
        for i in range(len(tokens)):
            # flatten out the multiple samples of the same schema
            mpath_tokens = tokens[i].view(-1, tokens[i].size(2)) # [M*D, L+1]
            mpath_edge_tokens = edge_tokens[i].view(-1, edge_tokens[i].size(2)) # [M*D, L]

            mpath_inputs, mpath_edge_inputs = self.embed_and_scale(mpath_tokens, mpath_edge_tokens, schemas[i])

            mpath_hidden_all = self.rotational_encoding(mpath_inputs, mpath_edge_inputs) # [M*D, L+1, H]

            # the first element in the sequence is the target node, the rest are transformed embeddings for other nodes in the metapath
            mpath_hidden_all = torch.cat([mpath_hidden_all[:, 0, :].unsqueeze(1).repeat(1, mpath_hidden_all.size(1) - 1, 1), mpath_hidden_all[:, 1:, :]], dim=2) # [M*D, L, 2H]
            mpath_hidden = torch.relu(self.intra_attention(mpath_hidden_all, dyn_context=intra_context)) # [M*D, 2H]

            # aggregate transformed embeddings from multiple samples of the same schema
            mpath_hidden = torch.sum(mpath_hidden.view(tokens[i].size(0), tokens[i].size(1), -1), dim=0) # [D, 2H]
            hidden.append(mpath_hidden)
        hidden = torch.stack(hidden, dim=1) # [D, N, 2H]

        # perform inter-metapath aggregation across transformed embeddings for each schema
        hidden_act = torch.mean(torch.tanh(self.dropout(self.inter_fc(hidden))), dim=0).expand_as(hidden) # [D, N, 2H]
        context = self.inter_context.unsqueeze(0).repeat(hidden_act.size(0), 1).unsqueeze(2) if inter_context is None else inter_context.unsqueeze(2)
        scores = torch.bmm(hidden_act, context) # [D, N, 1]

        outputs = torch.sum(hidden * scores, dim=1) # [D, 2H]
        outputs = self.dropout(self.output_fc(outputs)) # [D, H]

        return outputs

class MatchNet(torch.nn.Module):
    def __init__(self, hidden_size, num_labels, drop=0.1):
        super().__init__()

        self.match_lstm = LstmNet(hidden_size)
        self.match_attn = AttnNet(hidden_size, drop=drop)
        self.match_fc = torch.nn.Linear(2 * hidden_size, num_labels)

        self.dropout = torch.nn.Dropout(drop)

    def forward(self, fact_inputs, sec_inputs, context=None): # [D, H], [C, H]
        sec_inputs = sec_inputs.expand(fact_inputs.size(0), sec_inputs.size(0), sec_inputs.size(1)) # [D, C, H]

        sec_hidden_all = self.match_lstm(sec_inputs) # [D, C, H]
        sec_hidden = self.match_attn(sec_hidden_all, dyn_context=context) # [D, H]

        logits = self.dropout(self.match_fc(torch.cat([fact_inputs, sec_hidden], dim=1))) # [D, C]
        scores = torch.sigmoid(logits).detach() # [D, C]
        return logits, scores


In [ ]:
import torch




# from model.submodules import HierAttnNet, MetapathAggrNet, MatchNet


class LeSICiN(torch.nn.Module):
    def __init__(self, hidden_size, num_labels, node_vocab_size, edge_vocab_size, vocab_size=None, label_weights=None, pthresh=0.55, lambdas=(0.5, 0.5), thetas=(3, 2, 3), drop=0.1):
        super().__init__()

        self.text_encoder = HierAttnNet(hidden_size, vocab_size=vocab_size)
        self.graph_encoder = MetapathAggrNet(node_vocab_size, edge_vocab_size, hidden_size)
        self.match_network = MatchNet(hidden_size, num_labels)

        self.match_context_transform = torch.nn.Linear(hidden_size, hidden_size)
        self.intra_context_transform = torch.nn.Linear(hidden_size, 2 * hidden_size) # We need double the hidden size for Struct Encoder dynamic context
        self.inter_context_transform = torch.nn.Linear(hidden_size, 2 * hidden_size)

        self.criterion = torch.nn.BCEWithLogitsLoss(pos_weight=label_weights)

        self.pred_threshold = pthresh
        self.lambdas = lambdas # weights for scores
        self.thetas = thetas # weights for losses
        self.dropout = torch.nn.Dropout(drop)

    def calculate_losses(self, logits_list, labels):
        loss = 0
        for i, logits in enumerate(logits_list):
            if logits is not None:
                loss += self.thetas[i] * self.criterion(logits, labels)
        return loss

    def forward(self, fact_batch, sec_batch, pthresh=None): # We have D documents in fact_batch and C sections in sec_batch
        if pthresh is not None:
            self.pred_threshold = pthresh

        # Encode fact text using HAN
        if not fact_batch.sent_vectorized:
            fact_attr_hidden = self.text_encoder(tokens=fact_batch.tokens, mask=fact_batch.mask) # [D, H]
        else:
            fact_attr_hidden = self.text_encoder(doc_inputs=fact_batch.doc_inputs, mask=fact_batch.mask) # [D, H]

        # Encode sec text using HAN
        if not sec_batch.sent_vectorized:
            sec_attr_hidden = self.text_encoder(tokens=sec_batch.tokens, mask=sec_batch.mask) # [C, H]
        else:
            sec_attr_hidden = self.text_encoder(doc_inputs=sec_batch.doc_inputs, mask=sec_batch.mask) # [C, H]

        # context vector for matching with fact attributes
        attr_match_context = self.dropout(self.match_context_transform(fact_attr_hidden)) # [D, H]

        # Attribute scores
        attr_logits, attr_scores = self.match_network(fact_attr_hidden, sec_attr_hidden, context=attr_match_context)

        # sec-side context vectors for Struct Encoder
        sec_intra_context = self.dropout(self.intra_context_transform(sec_attr_hidden)).repeat(sec_batch.num_mpath_samples, 1) # [M*C, H]
        sec_inter_context = self.dropout(self.inter_context_transform(sec_attr_hidden)) # [C, H]

        # Encode sec graph using MAGNN
        sec_struct_hidden = self.graph_encoder(sec_batch.node_tokens, sec_batch.edge_tokens, sec_batch.schemas, intra_context=sec_intra_context, inter_context=sec_inter_context) # [C, H]

        # Alignment scores
        align_logits, align_scores = self.match_network(fact_attr_hidden, sec_struct_hidden, context=attr_match_context)

        if fact_batch.sample_metapaths:
            # fact-side context vectors for Struct Encoder
            fact_intra_context = self.dropout(self.intra_context_transform(fact_attr_hidden)).repeat(fact_batch.num_mpath_samples, 1) # [M*D, H]
            fact_inter_context = self.dropout(self.inter_context_transform(fact_attr_hidden)) # [D, H]

            # Encode sec graph using MAGNN
            fact_struct_hidden = self.graph_encoder(fact_batch.node_tokens, fact_batch.edge_tokens, fact_batch.schemas, intra_context=fact_intra_context, inter_context=fact_inter_context) # [D, H]

            # context vector for matching with fact structure
            struct_match_context = self.dropout(self.match_context_transform(fact_struct_hidden)) # [D, H]

            # Structural scores
            struct_logits, struct_scores = self.match_network(fact_struct_hidden, sec_struct_hidden, context=struct_match_context)

        else:
            struct_logits = None

        # Combine scores and losses
        scores = (self.lambdas[0] * attr_scores + self.lambdas[-1] * align_scores)
        predictions = (scores > self.pred_threshold).float()

        if fact_batch.annotated:
            loss = self.calculate_losses([attr_logits, struct_logits, align_logits], fact_batch.labels)
        else:
            loss = None

        return loss, predictions


In [ ]:
import torch
import torch_sparse
from tqdm import tqdm
from collections import defaultdict
import os
import pickle as pkl
# from data_helper import MiniBatch

# Create word vocab (if not sent vectorized) and label vocab from the specific order in the Section dataset
# def generate_vocabs(train_data, label_data, limit=30000, thresh=1):
#     if not train_data.sent_vectorized:
#         freqs = defaultdict(int)
#         for instance in tqdm(train_data.dataset + label_data.dataset, desc="Creating vocabulary"):
#             for sent in instance['text']:
#                 for word in sent:
#                     freqs[word] += 1
#         vocab_set = set(w for w, f in freqs.items() if f >= thresh)
#         vocab = {k: i for i, k in enumerate(vocab_set)}
#     else:
#         vocab = None

#     label_vocab = {}
#     for instance in tqdm(label_data.dataset, desc="Creating label vocabulary"):
#         label_vocab[instance['id']] = len(label_vocab)

#     return vocab, label_vocab

def generate_vocabs(train_data, label_data, limit=30000, thresh=1, label_vocab_file='/kaggle/input/ipc-recommandation/label_vocab.json'):
    # Ensure thresh has a valid numeric value.
    # if thresh is None:
    #     thresh = 1

    # Compute word vocabulary only if not already sentence vectorized.
    if not train_data.sent_vectorized:
        freqs = defaultdict(int)
        for instance in tqdm(train_data.dataset + label_data.dataset, desc="Creating vocabulary"):
            for sent in instance['text']:
                for word in sent:
                    freqs[word] += 1
        vocab_set = set(w for w, f in freqs.items() if f >= thresh)
        vocab = {k: i for i, k in enumerate(vocab_set)}
    else:
        vocab = None

    # If a label_vocab file is provided and exists, load it; otherwise, compute it.
    # if label_vocab_file is not None and os.path.exists(label_vocab_file):
    #     with open(label_vocab_file, 'r') as fr:
    #         label_vocab = pkl.load(fr)
    # else:
    label_vocab = {}
    for instance in tqdm(label_data.dataset, desc="Creating label vocabulary"):
        label_vocab[instance['id']] = len(label_vocab)

    return vocab, label_vocab


# Create the entire graph by combining the label tree and fact-sec citation network
def generate_graph(label_vocab, type_map, label_tree_edges, cit_net_edges, label_name='section'):
    node_vocab = defaultdict(dict) # each key is a node_type, and each value is a dict storing the vocab of all nodes under given node type
    node_vocab[label_name] = label_vocab # manually set this since we want the label vocab to be consistent with node vocab for labels

    edge_vocab = {}
    edge_indices = defaultdict(list) # each key is a tuple (src node type, relationship name, trg node type), and each value is a list storing the edges from src node type to trg node type

    for (node_a, edge_type, node_b) in label_tree_edges + cit_net_edges:
        # first get the node type
        node_a_type, node_b_type = type_map[node_a], type_map[node_b]

        # create new vocab entries for edges and nodes
        if edge_type not in edge_vocab:
            edge_vocab[edge_type] = len(edge_vocab)

        if node_a not in node_vocab[node_a_type]:
            node_vocab[node_a_type][node_a] = len(node_vocab[node_a_type])
        if node_b not in node_vocab[node_b_type]:
            node_vocab[node_b_type][node_b] = len(node_vocab[node_b_type])

        # get node indices
        node_a_token = node_vocab[node_a_type][node_a]
        node_b_token = node_vocab[node_b_type][node_b]

        edge_indices[(node_a_type, edge_type, node_b_type)].append([node_a_token, node_b_token])

    num_nodes = {ntype: len(nodes) for ntype, nodes in node_vocab.items()}

    # same as edge_indices except that the edges under each key are now stored as sparse matrices
    adjacency = {}
    for keys, edges in edge_indices.items():
        row, col = torch.tensor(edges).t()
        sizes = (num_nodes[keys[0]], num_nodes[keys[-1]])
        adj = torch_sparse.SparseTensor(row=row, col=col, sparse_sizes=sizes)
        adjacency[tuple(keys)] = adj

    return node_vocab, edge_vocab, edge_indices, adjacency

# create label weights for BCE Loss since we have unbalanced class distribution
def generate_label_weights(train_data, label_vocab, dev='cuda:0', scheme="tws", thresh=10.):
    pos = torch.zeros(len(label_vocab), device=dev)
    for instance in tqdm(train_data, desc="Generating label weights"):
        for l in instance['labels']:
            pos[label_vocab[l]] += 1
    weights = torch.clamp(pos.max() / pos, max=thresh) if scheme == 'tws' else len(train_data) / pos
    return weights

# Unified code to deal with a single train / dev / test / inference pass over the dataset
def train_dev_passs(model, optimizer, fact_loader, sec_batch, metrics=None, pred_threshold=None, train=False, infer=False, label_vocab=False):

    model.train() if train else model.eval()

    if infer:
        outputs = []
        inv_label_vocab = {v: k for k, v in label_vocab.items()}

    for i, fact_batch in enumerate(tqdm(fact_loader, desc="Flowing data through model")):
        torch.cuda.empty_cache()

        loss, predictions = model(fact_batch.cuda(), sec_batch.cuda(), pthresh=pred_threshold)

        if train:
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        if not infer:
            batch_loss = loss.item()
            metrics(predictions, fact_batch.labels, loss=batch_loss)

        else:
            for i, instance_preds in enumerate(predictions):
                # gather true predictions
                pred_list_indices = torch.nonzero(instance_preds, as_tuple=False).squeeze(1)
                pred_list = [inv_label_vocab[idx.item()] for idx in pred_list_indices]
                outputs.append({'id': fact_batch.example_ids[i], 'predictions': pred_list})

    return metrics.calculate_metrics() if not infer else outputs

class MultiLabelMetrics(torch.nn.Module):
    def __init__(self, num_classes, dev='cuda', loss=True):
        super().__init__()

        self.match = torch.zeros(num_classes, device=dev) # count no. of true positives for each label
        self.predictions = torch.zeros(num_classes, device=dev) # count no. of true positives + false positives for each label
        self.labels = torch.zeros(num_classes, device=dev) # count no. of true positives + false negatives for each label
        self.run_jacc = 0 # running sum of jaccard scores
        self.counter = 0 # count no. of batches
        if loss:
            self.run_loss = 0 # running sum of losses

    # to be called with a batch of predictions and true labels
    def forward(self, predictions, labels, loss=None):
        match = predictions * labels # true positives for this batch

        # increment counts
        self.match += match.sum(dim=0)
        self.labels += labels.sum(dim=0)
        self.run_jacc += torch.sum(torch.logical_and(predictions, labels).sum(dim=1) / torch.logical_or(predictions, labels).sum(dim=1)).item()
        self.counter += 1

        if loss is not None:
            self.run_loss += loss

    # reset counters
    def refresh(self):
        self.match.fill_(0)
        self.predictions.fill_(0)
        self.labels.fill_(0)
        self.run_jacc = 0
        self.counter = 0
        if 'run_loss' in self.__dict__:
            self.run_loss = 0
        return self

    # calculate the metrics and return self
    def calculate_metrics(self, refresh=True):
        prec = self.match / self.predictions # P = TP / (TP + FP)
        rec = self.match / self.labels # R = TP (TP + FN)

        prec[prec.isnan()] = 0
        rec[rec.isnan()] = 0

        f1 = 2 * prec * rec / (prec + rec) # F1 = 2 * P * R / (P + R)
        f1[f1.isnan()] = 0

        # macro --> average across each label
        self.macro_prec = prec.mean().item()
        self.macro_rec = rec.mean().item()
        self.macro_f1 = f1.mean().item()

        match_total = self.match.sum().item()
        preds_total = self.predictions.sum().item()
        labels_total = self.labels.sum().item()

        # micro --> take total counts
        self.micro_prec = match_total / preds_total if preds_total > 0 else 0
        self.micro_rec = match_total / labels_total
        self.micro_f1 = 0 if self.micro_prec + self.micro_rec == 0 else 2 * self.micro_prec * self.micro_rec / (self.micro_prec + self.micro_rec)

        self.jacc = self.run_jacc / self.counter
        if 'run_loss' in self.__dict__:
            self.loss = self.run_loss / self.counter

        if refresh:
            self.refresh()

        return self


In [ ]:
print("\nPreparing PyTorch environment")
print("==================================================")

import torch
import random
from tqdm import tqdm
from functools import partial
import sent2vec
from gensim.models import KeyedVectors
import json

In [ ]:
def run_ai(message):
    torch.autograd.set_detect_anomaly(True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    
    # SEED = hc['seed']
    
    # random.seed(SEED)
    # np.random.seed(SEED)
    # torch.manual_seed(SEED)
    # torch.cuda.manual_seed(SEED)
    
    if dc['s2v_path'] is not None:
        sent2vec_model = sent2vec.Sent2vecModel()
        sent2vec_model.load_model(dc['s2v_path'])
    
    dataset = []
    
    # if hc['do_infer']:
    message1 = ["Prosecution case in brief was that Tuliya Devi, daughter of informant Bhullu Rajbhar (PW-1) was married to Gullu 5 years ago.", "After marriage Tuliya Devi was treated with cruelty by her husband Gullu, father-in-law Bal Chand and mother-in-law Ramwati for demand of dowry.", "For this reason these three accused had committed murder of Tuliya Devi on 8.5.2010 by inflicting injuries on her body at their house situate in village Chaubepur, police station- Mardah, district Ghazipur and tried to dispose of her dead body.", "After receiving knowledge of this incident, victim's father Bhullu (informant) had given a written report (Ex-Ka-1) to the police on the basis of which case crime no. 689 of 2010 was registered.", "3.In the aforesaid case inquest of the dead body was performed on 08.05.2010 and postmortem of the deceased was conducted the same evening at 9:50 pm.", "In the postmortem report several lacerated wounds and other marks of injuries were found on the dead body of the deceased.", "In postmortem (Ex-Ka-1) the doctor had opined that cause of death was haemorrhage and shock as a result of the said ante-mortem injuries.", "In this report approximate time of death was reported about 1 day from the time of postmortem.", "After completion of investigation police had submitted charge-sheet against three accused Gullu, Bal Chand and Ramwati for offences under section 498-A, 304-B IPC and under section 3/4 Dowry Prohibition Act.", "Accused had pleaded not guilty and claimed to be tried.", "5.In support of charges prosecution side examined PW-1, Bhullu (informant), PW-2 Barmati (mother of the deceased), PW-3 - Constable Sunil Kumar Singh (who prepared chik FIR and registerd case), PW-4 - Dr. Krishna Kumar Verma (who performed the postmortem), PW-5 Vinay Kumar Rai, Naib Tehsildar (for inquest report) and PW-6 Chrinjeev Nath Sinha (Investigation Officer).", "Hon'ble Pramod Kumar Srivastava,J.", "(Delivered by Hon'ble Pramod KumarSrivastava, J.)", "1.This appeal has been preferred against the conviction and punishment dated 28.2.2012 passed in State Vs.", "Gullu and others relating to Case Crime No. 689 of 2010, police station Mardah, district Ghazipur by which three accused Gullu, Bal Chand and Smt .", "Ramwati were convicted for charge under section 302/34 IPC and were punished with imprisonment for life and fine of Rs. 5000/- (in default of payment three months simple imprisonment).", "4.During trial all the accused were charged for offences under section 498-A, 304 B IPC and under section 3/4 Dowry Prohibition Act, alongwith alternative charge framed later on for offence under section 302 IPC.", "6.After closure of prosecution evidence statement of accused were recorded in which they denied the facts of charge as well as evidence adduced against them, without any specific averments.", "Defence side had examined DW-1 Madan, r/o village Chaubeypur.", "7.After receiving evidence from both the sides and after affording opportunity of hearing as well as considering the argument of the parties, learned Addl.", "Sessions Judge passed judgment dated 28.2.2012 by which all the accused were acquitted from the charges under section 498-A, 304-B IPC and 3/4 Dowry Prohibition Act, but were convicted for the charge under section 302-IPC.", "Thereafter the trial court had afforded an opportunity of hearing on the point of quantum of sentence to the accused and passed orders of punishment as above.", "Aggrieved by this judgment dated 28.2.2012 all the three accused have preferred the present appeal.", "8.Sri Sudist and Sri Janardan Singh Yadav appeared for appellants; and State was represented by Mrs. Usha Kiran, AGA.", "9.A perusal of evidence adduced during trial indicates that during post-mortem following ante-mortem injuries were found on the dead body of the deceased Tuliya Devi that were caused or occurred approximately at the time mentioned in the charge, i.e., anytime in the night of 8/9-5-2010:", "1.Lacerated wound left side chin 6 cm x 1 cm mussel deep chin.", "2.Lacerated wound from right shoulder to just below right elbow 35 cm x 2 cm x bone deep.", "3.Abrasion 5cm x 2 cm just laterac of left eye.", "4.Abrasion on top of right shoulder 8 cm x 4 cm.", "5.Abrasion 20cm x 10 cm right side chest.", "6.brasion 8 cm x 2 cm right iliac chest.", "7.Lacerated wound 5 cm x 3 cm at middle of front of right of leg underlying bond was fractured.", "8.Lacerated wound 6 cm x 3 cm just below right knee.", "9.Lacerated wound 8 cm x 3 cm below front of left kneel under lying bone was fractured.", "10.PW-4 the doctor reported that the cause of death of victim-deceased Tuliya Devi was shock and haemmerhage due to the above mentioned ante-mortem injuries.", "Although the defence had adduced one witness to indicate that the cause of death of the deceased may be accidental falling from the roof, and DW-1 Madan was examined in this regard but these facts could not be substantiated in the light of available evidence and circumstances.", "11.A perusal of the impugned judgment reveals that this finding of learned Additional Sessions Judge is correct that deceased Tuliya Devi had died due to injuries found on her body in the house of the accused-appellants at about the time mentioned in the charge, and the accused-appellants were responsible for causing such injuries to Tuliya Devi.", "In these circumstances, the trial court had rightly reached to the conclusion that due to the above mentioned deliberate caused injuries Tuliya Devi died, and accused-appellants are responsible for inflicting those homicidal injuries.", "12.Learned Additional Sessions Judge had considered facts and circumstances including evidence adduced and reached to the conclusion that though there is no conclusive evidence relating to dowry death and demand of dowry, but injuries found on body of the deceased were not accidental.", "Trial Court found that those injuries were homicidal, for inflicting of which accused persons were responsible, because it were only they who were present in their house when such injuries had occurred on the body of the deceased Tuliya Devi.", "13.Learned counsel appearing for the appellants fairly admitted the contents of facts relating to charge, namely victim Tuliya Devi having succumbed to the injuries found on her body in the house of appellants."]
    
    data_dict = [{
        'id': 1001,
        'text': message
        },
    ]
    infer_dataset = LSIDataset(data_list = data_dict)
    infer_dataset.preprocess()
    # print(infer_dataset.dataset)
    infer_dataset.sent_vectorize(sent2vec_model)
    infer_dataset.save_data(dc['infer_cache'])
    dataset = infer_dataset

    print("\nPreparing Datasets")
    print("==================================================")
    # sec_dataset = LSIDataset.load_data(dc['dev_cache'])
    sec_dataset = LSIDataset(jsonl_file=dc['sec_src'])
    sec_dataset.preprocess()
    sec_dataset.sent_vectorize(sent2vec_model)
    sec_dataset.save_data(dc['sec_cache'])
    
    vocab, label_vocab = generate_vocabs(dataset, sec_dataset, limit=hc['vocab_limit'], thresh=hc['vocab_thresh'])
    
    with open(dc['type_map']) as fr:
        type_map = json.load(fr)
    with open(dc['label_tree']) as fr:
        label_tree = json.load(fr)
    with open(dc['citation_network']) as fr:
        citation_net = json.load(fr)
    with open(dc['schemas']) as fr:
        schemas = json.load(fr)
    for sch in schemas.values():
        for path in sch:
            for i, edge in enumerate(path):
                path[i] = tuple(path[i])
    
    node_vocab, edge_vocab, edge_indices, adjacency = generate_graph(label_vocab, type_map, label_tree, citation_net)
    # sec_weights = generate_label_weights(dataset, label_vocab)
    
    L = len(label_vocab)
    N = {k: len(v) for k,v in node_vocab.items()}
    E = len(edge_vocab)
    
    sec_loader = torch.utils.data.DataLoader(
        sec_dataset,
        batch_size=len(label_vocab),
        collate_fn=partial(
            collate_func,
            schemas=schemas['section'],
            type_map=type_map,
            node_vocab=node_vocab, 
            edge_vocab=edge_vocab,
            adjacency=adjacency,
            max_segments=hc['max_segments'],
            max_segment_size=hc['max_segment_size'],
            num_mpath_samples=hc['num_mpath_samples']
            ),
        pin_memory=True,
        num_workers=2
    )
    for sec_batch in sec_loader:
        break
    
    lsc_model = LeSICiN(
        hc['hidden_size'],
        L,
        N,
        E,
        lambdas=hc['lambdas'],
        thetas=hc['thetas'],
        pthresh=hc['pthresh'],
        drop=hc['dropout']
        ).cuda()
    
    if dc['model_load'] is not None:
        lsc_model.load_state_dict(torch.load(dc['model_load'], map_location='cuda'), strict=False)
        print("loaded")
    
    optimizer = torch.optim.AdamW(lsc_model.parameters(), lr=hc['opt_lr'], weight_decay=hc['opt_wt_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=hc['sch_patience'], factor=hc['sch_factor'])
    
    infer_loader = torch.utils.data.DataLoader(
    infer_dataset,
    batch_size=hc['infer_bs'],
    collate_fn=partial(
        collate_func,
        label_vocab=label_vocab,
        max_segments=hc['max_segments'],
        max_segment_size=hc['max_segment_size']
        ),
    pin_memory=True,
    num_workers=2
    )
    infer_outputs =  train_dev_passs(lsc_model, optimizer, infer_loader, sec_batch, infer = True, pred_threshold=hc['pthresh'], label_vocab=label_vocab)
    return infer_outputs[0]['predictions']

In [ ]:

# from model.model import *
# from data_helper import *
# from helper import *



    # infer_dataset = LSIDataset.load_data(dc['infer_cache'])

In [ ]:
!pip install flask
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2tNAC56VtjmtEsPPA9axlMBf3I4_2nyiQuQCiHCv982guHXCj")


In [ ]:
print("\nPreparing Model")
print("==================================================")

# best_model = torch.load('/kaggle/input/ipc-recommandation/best_model.pt')
# train_dev_pass(model, optimizer, fact_loader, sec_batch, metrics=None

In [ ]:
!pip install pyngrok
!pip install cors

!pip install flask-cors
!ngrok config add-authtoken 2tNAC56VtjmtEsPPA9axlMBf3I4_2nyiQuQCiHCv982guHXCj

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Load your AI model here (replace with your actual model loading code)
def load_model():
    # Example: model = torch.load('model.pth')
    return "Your model loaded successfully!"

model = load_model()

# Define inference function
def run_inference(input_data):
    # Replace with your actual inference logic
    # Example: prediction = model.predict(input_data)
    return f"Prediction for {input_data}"

# API endpoint
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    input_data = data.get('input')
    print(input_data)
    # input_case = json.loads(input_data)
    print(type(input_data))
    input_sents = input_data[0].split('.')
    print(input_sents)
    result = run_ai(input_sents)
    # result = "hello world"
    return jsonify({"prediction": result})


if __name__ == '__main__':
    # Start Flask server on port 5000
    public_url = ngrok.connect(5000).public_url
    print(f" * Public URL (valid for 1 hour): {public_url}/predict")
    app.run(host='0.0.0.0', port=5000)

In [ ]:
def test(input_data):
    # data = request.json
    # input_data = data.get('input')
    input_case = json.loads(input_data)
    input_sents = input_data[0].split('.')
    result = run_ai(input_sents)
    print(result)
    return jsonify({"prediction": result})


In [ ]:
# ppsent = message.strip().lower().translate(str.maketrans('', '', string.punctuation))
# print(ppsent)
data = [
    "He killed someone",
    "He stabbed him in the eye infront of me",
    "He then ran away with the knife"
]

json_string = json.dumps(data)
print(json_string)

In [ ]:
output =  test(json_string)
output

# if hc['do_infer']:



In [ ]:
infer_outputs =  train_dev_passs(lsc_model, optimizer, infer_loader, sec_batch, infer = True, pred_threshold=hc['pthresh'], label_vocab=label_vocab)
# print("TEST Results || %.4f | %.4f %.4f %.4f" % (test_mlmetrics.loss, test_mlmetrics.macro_prec, test_mlmetrics.macro_rec, test_mlmetrics.macro_f1))

In [ ]:
print(infer_outputs)

In [ ]:
target_ids = [infer_outputs[i]['id'] for i in range(len(infer_outputs))]

In [ ]:
# with open('/content/output.txt', 'w') as fw:
#     fw.write('\n'.join([json.dumps(doc) for doc in infer_outputs]))
data = []
with open('/kaggle/input/ipc-recommandation/test.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # Remove any extra whitespace and skip empty lines
        line = line.strip()
        if line:
            data.append(json.loads(line))



In [ ]:
real_dict = {item['id']: item for item in data}
predicted_dict = {item['id']: item for item in infer_outputs}

# Get all unique IDs (combines IDs from both datasets)
all_ids = set(real_dict.keys()).union(set(predicted_dict.keys()))

# Print header
print(f"{'ID':<10} {'Real_IPC':<15} {'Predicted_IPC':<15}")
print("-" * 40)

# Print each row sorted by ID (optional: remove sorted() if order doesn't matter)
for i, id in enumerate(sorted(all_ids)):
    if (i > 10):
      break
    real_ipc = real_dict.get(id, {}).get('labels', 'N/A')  # Use 'N/A' if missing
    predicted_ipc = predicted_dict.get(id, {}).get('predictions', 'N/A')
    if isinstance(real_ipc, list):
        real_ipc = ', '.join(real_ipc)
    if isinstance(predicted_ipc, list):
        predicted_ipc = ', '.join(predicted_ipc)

    print(f"{id:<10} \|{real_ipc:<15}\| {predicted_ipc:<15}")
    print(f"{id:<10} \|{real_ipc:<15}\| {predicted_ipc:<15}")